In [1]:
import pandas as pd

# Завантаження даних
url = "https://raw.githubusercontent.com/susanli2016/Machine-Learning-with-Python/master/data/renfe_small.csv"
df = pd.read_csv(url)

# Виведення перших 5 рядків
print(df.head())


           insert_date    origin destination           start_date  \
0  2019-04-22 08:00:25    MADRID     SEVILLA  2019-04-28 08:30:00   
1  2019-04-22 10:03:24    MADRID    VALENCIA  2019-05-20 06:45:00   
2  2019-04-25 19:19:46    MADRID     SEVILLA  2019-05-29 06:20:00   
3  2019-04-24 06:21:57   SEVILLA      MADRID  2019-05-03 08:35:00   
4  2019-04-19 21:13:55  VALENCIA      MADRID  2019-05-10 09:40:00   

              end_date train_type  price         train_class      fare  
0  2019-04-28 11:14:00      ALVIA    NaN             Turista  Flexible  
1  2019-05-20 08:38:00        AVE  21.95             Turista     Promo  
2  2019-05-29 09:16:00    AV City  38.55             Turista     Promo  
3  2019-05-03 15:26:00      LD-MD  34.35  Turista con enlace   Promo +  
4  2019-05-10 11:20:00        AVE  45.30             Turista     Promo  


In [2]:
# Перевірка наявності пропущених значень
print(df.isnull().sum())

# Видалення рядків з пропущеними значеннями
df.dropna(inplace=True)

# Перевірка унікальних значень у стовпці 'price'
print(df['price'].unique())


insert_date       0
origin            0
destination       0
start_date        0
end_date          0
train_type        0
price          3082
train_class     103
fare            103
dtype: int64
[ 21.95  38.55  34.35  45.3   85.1   98.    33.65  43.25  76.3   61.45
  54.45 102.15 100.4   66.55  58.95  60.3  107.7   28.35  47.3  128.5
  75.4   62.2   43.55 143.4   66.75  39.45  71.    80.15  28.05  57.75
  49.15  45.8   57.    73.1   78.8   54.1   55.4   53.4   51.15  47.
  69.4   27.8   43.75  58.15  85.08  22.9   90.5   35.15  49.55  33.5
  28.1   25.75  69.8   40.95  58.45  29.9   79.65  38.75  20.6   88.95
  94.55  46.8  129.3  115.65  59.1   46.15  59.6   64.05  82.35  87.4
  42.75 181.5   55.8   32.9   59.5   40.3  127.1   16.6   52.45  31.2
  39.9   44.7   52.5   34.2  120.5  127.05  41.2  214.2   30.05  91.5
  34.65 112.55  47.05  98.01  62.4   17.15  68.65  35.5  169.2   69.35
  36.15  36.05  21.05  21.65  72.3   67.2  140.   101.5   56.65  32.4
  32.45  38.8   40.6   43.7   21.4

In [3]:
# Створення категорій цін
def categorize_price(price):
    if price < 50:
        return 'low'
    elif 50 <= price < 100:
        return 'medium'
    else:
        return 'high'

df['price_category'] = df['price'].apply(categorize_price)

# Перевірка розподілу нової змінної
print(df['price_category'].value_counts())


price_category
medium    12559
low        7815
high       2342
Name: count, dtype: int64


In [4]:
from sklearn.preprocessing import LabelEncoder

# Кодування категоріальних змінних
categorical_cols = ['train_type', 'origin', 'destination']
le = LabelEncoder()

for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

# Кодування цільової змінної
df['price_category'] = le.fit_transform(df['price_category'])


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# Визначення ознак та цільової змінної
X = df[['train_type', 'origin', 'destination']]
y = df['price_category']

# Розділення на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ініціалізація та тренування моделі
model = GaussianNB()
model.fit(X_train, y_train)

# Прогнозування та оцінка моделі
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.29      0.93      0.44       471
           1       0.85      0.41      0.56      1576
           2       0.60      0.54      0.57      2497

    accuracy                           0.54      4544
   macro avg       0.58      0.63      0.52      4544
weighted avg       0.65      0.54      0.55      4544

